### User Input

In [ ]:
pruning_user_params = {
    'model_depth': 3,       # 3, 5, 7
    'prune layers': 'dense', # none, all, dense, or conv
    'verbose_level': 0
}

### Path Setup

In [ ]:
model_name = 'paramcount1735975.h5'

mfccs_json_path = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale_emptyocean.json"

saved_model_path = f'/Users/seantedesco/Documents/marine-mammal-call-classification/saved_model/layers3/{model_name}'

pruned_model_path = f'/Users/seantedesco/Documents/marine-mammal-call-classification/saved_model/pruned_models/layers3/{model_name}'

### Imports

In [ ]:
import os
import tempfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import matplotlib.pyplot as plt

### Load in the MFCC File

In [ ]:
from compression_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

### Prepare the Datasets

In [ ]:
# create train, validation and test sets
from compression_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.6, 0.5) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Load in Saved Models

In [ ]:
loaded_model = tf.keras.models.load_model(saved_model_path)

### Generate List of Pruned Models

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(loaded_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

### Train the Pruned Models

In [ ]:
callbacks = [tfmot.sparsity.keras.UpdatePruningStep(),]

model_for_pruning.fit(X_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(X_test, y_test, verbose=0)
print('Pruned test accuracy:', model_for_pruning_accuracy)

### Save the Pruned Models

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
tf.keras.models.save_model(model_for_export, pruned_model_path, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_model_path)

### Get List of Pruned Model Memory Size

In [ ]:
from compression_lib import get_gzipped_model_size
pruned_byte_count = get_gzipped_model_size(pruned_model_path)
saved_byte_count = get_gzipped_model_size(saved_model_path)
print("Size of gzipped pruned Keras model: %.2f bytes" % (pruned_byte_count))

In [ ]:
print(f'{saved_byte_count},{pruned_byte_count},{model_for_pruning_accuracy}')

# copy this into a CSV file

### Introduce Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = f'/Users/seantedesco/Documents/marine-mammal-call-classification/saved_model/pruned_and_quantized_models/{model_name}.tflite'

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)
print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(X_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()
  return accuracy

interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)




print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(saved_model_path)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_model_path)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

print('Pruned test accuracy:', model_for_pruning_accuracy)
print('Pruned and quantized test accuracy:', test_accuracy)